In [ ]:
%%time

# -*- coding: utf-8 -*-

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup as bs
import pandas as pd
import time
import re
from tqdm import tqdm


def izkor_scraper():
    
    DRIVER_PATH = r'C:\Users\zeevs\Downloads\chromedriver.exe'

    options = Options()
    options.headless = True
    options.add_argument("--window-size=1920,1200")
    options.add_argument("--dns-prefetch-disable")

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.delete_all_cookies()

    ### Opening the website ###
    driver.get('https://www.izkor.gov.il/search-by-date/date/06-10-1973/31-05-1974/d')
    time.sleep(10)

    ### Click the button for more results ###
    button_click = driver.find_element(By.CLASS_NAME, 'btn.btn-success.more-results-btn.ng-star-inserted')
    [(button_click.send_keys(Keys.ENTER), time.sleep(5)) for x in tqdm(range(0,120))] 
    print('Done!')

    soup = bs(driver.page_source, 'html.parser')
    # soup
    soup_string = soup.find_all(name='search-result-person')
    
    return soup_string


string = izkor_scraper()

In [ ]:
%%time


def soliders_url_list():

    text_to_remove = ['>','</h5>','%"%',']',r'\([^()]*\)']

    people_n_id = []

    for x in tqdm(string):

        parents_start = x.text.find('בן')
        parents_end = x.text.find('נפל')
        photo_start = str(x).find('src')+5
        photo_end = str(x).find('jpg')+3
        rank_start = str(x).find('<span')+37
        rank_end = str(x).find('</span>')
        people_n_id.append([x.text[parents_start:parents_end],
                            x.attrs['id'],
                            ''.join([str(x)[rank_start:rank_end],str(x.select('h5'))[65:]]),
                            str(x)[photo_start:photo_end],
                           ])


    for x in tqdm(people_n_id):
        x[2]=re.sub('|'.join(text_to_remove).strip(), '', x[2])
        x[2]=x[2].strip()
        x.append(f'https://www.izkor.gov.il/{x[2]}/{x[1]}')
        
    return people_n_id


people_list = soliders_url_list()
people_list[0]

In [ ]:
%%time

def soliders_df(): ### df from the data scraped from Izkor website - extracting it to csv ###
                   ### and adding it to the file I recieved from Yom kippur organization ###

    data = pd.DataFrame(people_list)
    data.rename(columns ={
        0: 'Parents',
        1: 'page_id',
        2: 'solider_name',
        3: 'url',
        4: 'photo'
    }, inplace = True)

    return data

df = soliders_df()
# df
df.to_csv(r'C:\Users\zeevs\Desktop\izkor_url.csv',encoding='utf-8-sig')

In [ ]:
%%time

import tkinter as tk
from tkinter import filedialog as fd

### after finishing working on the file, I generated a list of lists ### 

def load_file():
    
    root = tk.Tk()
    name = fd.askopenfilename() 
    root.destroy()
    
    return name


def get_soliders_list(): ### generating a list of lists (list for each day)
        
    df = pd.read_excel(load_file(), sheet_name='final_list',engine='openpyxl') #engine='pyxlsb')
    df_list = [df[df.birthday == birthday].values.tolist() for birthday in tqdm(df.birthday.unique())]    
        
    return df_list

soliders_list = get_soliders_list()  

### Example for what the list per day looks like ###
# soliders_list[47]

In [ ]:
%%time

import datetime as dt
import os
from pptx import Presentation 
from pptx.util import Inches, Pt
from pptx.dml.color import ColorFormat, RGBColor
from pptx.enum.text import PP_ALIGN
from pptx.enum.lang import MSO_LANGUAGE_ID
from pptx.enum.action import PP_ACTION
import aspose.slides as a_slides
from requests_tor import RequestsTor
import win32com.client

os.environ["NLS_LANG"] = ".UTF8" 
x = dt.datetime.now()

requests = RequestsTor(tor_ports=(9000, 9001, 9002, 9003, 9004), autochange_id=1)

In [ ]:
%%time

def create_slide_show():

    #     """ Ref for slide types: 
    #     0 ->  title and subtitle
    #     1 ->  title and content
    #     2 ->  section header
    #     3 ->  two content
    #     4 ->  Comparison
    #     5 ->  Title only 
    #     6 ->  Blank
    #     7 ->  Content with caption
    #     8 ->  Pic with caption
    #     """

    # Creating slide object to add 
    # in ppt i.e. Attaching slides 
    # with Presentation i.e. ppt
    # slide = root.slides.add_slide(first_slide_layout)

    for birthday in tqdm(soliders_list): ### Createing the main title slide ###
        # Creating an Presentation object
        ppt = Presentation() 
        ppt.slide_height = Inches(9)   
        ppt.slide_width = Inches(16) 


        blank_slide_layout = ppt.slide_layouts[6] 
        slide = ppt.slides.add_slide(blank_slide_layout) 

        ### Background ###
        background = slide.shapes.add_picture(r'C:\Users\zeevs\Desktop\kipur\bg4.jpg',
                                          left=0, 
                                          top=0, 
                                          width=ppt.slide_width,
                                          height=ppt.slide_height)

        ### War Sign ###
        sign = slide.shapes.add_picture(r'C:\Users\zeevs\Desktop\kipur\sign.jpg',
                                    Inches(1.03/2.54), # left
                                    Inches(0.72/2.54), # Top 
                                    Inches(9.5/2.54), #width
                                    Inches(2.96/2.54)) #height


        ### Candle ###
        candle = slide.shapes.add_picture(r'C:\Users\zeevs\Desktop\kipur\candle.gif',
                                    Inches(17.22/2.54), # left
                                    Inches(10/2.54), # Top 
                                    Inches(6.2/2.54), #width
                                    Inches(7.88/2.54)) #height


        first_slide_title = slide.shapes.add_textbox(Inches(0.77/2.54), # left
                                   Inches(5.8/2.54), # Top 
                                   Inches(39.1/2.54), # Width
                                   Inches(1.97/2.54) # height
                                    )


        fst = first_slide_title.text_frame
        fst.text = 'מיזם לציון יום הולדתם של חללי צה"ל במלחמת יום הכיפורים'
        fst.paragraphs[0].alignment = PP_ALIGN.CENTER
        fst.paragraphs[0].font.size = Pt(40)
        fst.paragraphs[0].font.bold = True
        fst.paragraphs[0].font.underline = True
        fst.paragraphs[0].font.language_id = MSO_LANGUAGE_ID.HEBREW


        bible = slide.shapes.add_textbox(Inches(15.92/2.54), # left
                                   Inches(18.94/2.54), # Top 
                                   Inches(8.8/2.54), # Width
                                   Inches(1.62/2.54) # height
                                    )

        bible_quote = bible.text_frame
        bible_quote.text = '''"כִּי מִדֵּי דַבְּרִי בּוֹ זָכֹר אֶזְכְּרֶנּוּ עוֹד"'''
        bible_quote.paragraphs[0].alignment = PP_ALIGN.RIGHT
        bible_quote.paragraphs[0].font.name = 'Aharoni'
        bible_quote.paragraphs[0].font.size = Pt(18)
        bible_quote.paragraphs[0].font.bold = True
        bible_quote_2 = bible_quote.add_paragraph()
        bible_quote_2.text = '(יט, ירמיהו לא) '
        bible_quote.paragraphs[1].alignment = PP_ALIGN.RIGHT
        bible_quote.paragraphs[1].font.size = Pt(14)
        bible_quote.paragraphs[1].font.bold = True
        bible_quote.paragraphs[1].font.name = 'Aharoni'


        for solider in birthday: ### Creating the slide for each solider ###

            # Selecting blank slide
            #     blank_slide_layout = ppt.slide_layouts[6] 

            #### Attaching slide to ppt ###
            slide = ppt.slides.add_slide(blank_slide_layout) 

            # For margins
            left = Inches(1) 
            right = Inches(2)
            height = Inches(1) 
            top = Inches(1)
            bottom = Inches(3)

            ### Background ###
            background = slide.shapes.add_picture(r'C:\Users\zeevs\Desktop\kipur\bg4.jpg',
                                                  left=0, 
                                                  top=0, 
                                                  width=ppt.slide_width,
                                                  height=ppt.slide_height)

            ### War Sign ###
            sign = slide.shapes.add_picture(r'C:\Users\zeevs\Desktop\kipur\sign.jpg',
                                            Inches(1.03/2.54), # left
                                            Inches(0.72/2.54), # Top 
                                            Inches(9.5/2.54), #width
                                            Inches(2.96/2.54)) #height

            ### Soliders images - Downloaded from the solider's page ###
            res = requests.get(solider[4]) 
            with open(''.join([solider[0],'.jpg']),'wb') as f:
                f.write(res.content)

            pic = slide.shapes.add_picture(''.join([solider[0],'.jpg']),
                                           Inches(29.72/2.54), # left
                                           Inches(8.99/2.54), # Top 
                                           Inches(5.29/2.54), # Width
                                           Inches(6.7/2.54) # height
                                          )

            ### creating textBox ###
            ### title ###
            slide_title = slide.shapes.add_textbox(Inches(5.3/2.54), # left
                                           Inches(5.06/2.54), # Top 
                                           Inches(30.04/2.54), # Width
                                           Inches(1.97/2.54) # height
                                            )

            st = slide_title.text_frame
            st.text = 'מיזם לציון יום הולדתם של חללי מלחמת יום הכיפורים'
            st.paragraphs[0].font.size = Pt(40)
            st.paragraphs[0].font.bold = True
            st.paragraphs[0].font.underline = True


            ### Solider Name ###
            solider_name = slide.shapes.add_textbox(Inches(3.92/2.54), # left
                                           Inches(13.93/2.54), # Top 
                                           Inches(25.17/2.54), # Width
                                           Inches(1.97/2.54) # height
                                            )
            sn = solider_name.text_frame
            sn.text = f'{solider[1]} ז"ל'
    #         sn.text = f'{solider[1]}'
            sn.paragraphs[0].alignment = a_slides.TextAlignment.RIGHT #PP_ALIGN.RIGHT
            sn.paragraphs[0].alignment = PP_ALIGN.RIGHT
            sn.paragraphs[0].font.size = Pt(40)
            sn.paragraphs[0].font.bold = True
            sn.paragraphs[0].font.underline = False
            sn.paragraphs[0].font.language_id = MSO_LANGUAGE_ID.HEBREW


            ### Birthday ###
            birthday = slide.shapes.add_textbox(Inches(18.75/2.54), # left
                                           Inches(11.77/2.54), # Top 
                                           Inches(10.06/2.54), # Width
                                           Inches(1.03/2.54) # height
                                            )

            bd = birthday.text_frame
            #     bd.text = f' הוא יום הולדתו של {str(x[2])}-היום ה'
            bd.text = solider[6]
            bd.paragraphs[0].alignment = PP_ALIGN.RIGHT
            bd.paragraphs[0].font.size = Pt(24)
            bd.paragraphs[0].font.bold = False
            bd.paragraphs[0].font.underline = False
            bd.paragraphs[0].font.language_id = MSO_LANGUAGE_ID.HEBREW



            ## Hyperlink ###
            link_to_page = slide.shapes.add_textbox(Inches(14.31/2.54), # left
                                           Inches(18.94/2.54), # Top 
                                           Inches(9.83/2.54), # Width
                                           Inches(1.03/2.54) # height
                                            )

            tf = link_to_page.text_frame
            r = tf.paragraphs[0].add_run()
            r.text = 'לסיפור חייו ולהדלקת נר לזיכרו'
            r.font.size = Pt(24)
            hlink = r.hyperlink
            hlink.address = solider[3] 


         ### save file ###
        if solider[2][3] == '0':
            ppt.save(''.join([r'G:\\My Drive\\idf_izkor\\',dt.datetime.strptime(solider[2][4], "%m").strftime("%B"),'-',solider[2][0:2],'.pptx']))
        else:
            ppt.save(''.join([r'C:\Users\zeevs\Desktop\izkor_',dt.datetime.strptime(str(solider[2][3:]), "%m").strftime("%B"),'-',solider[2][0:2],'.pptx']))

    return

create_slide_show()

In [ ]:
%%time

def file_path_list():
    
    ### List of ppt files ###
    folderpath = r'G:\My Drive\idf_izkor\\' # make sure to put the 'r' in front
    filepaths  = [os.path.join(folderpath, name) for name in tqdm(os.listdir(folderpath))]
    filepaths
    
    return filepaths


def design_slide_show(): ### designing the slides with transitions effects ### 

    license = a_slides.License()
    license.set_license(r'C:\Users\zeevs\Downloads\Aspose.Slides.Python.NET.lic')

    for slide_show in tqdm(file_paths):
     ### Instantiate Presentation class that represents a presentation file ###
        with a_slides.Presentation(slide_show) as pres:    

            pres.slides[0].slide_show_transition.advance_on_click = True
            pres.slides[0].slide_show_transition.advance_after_time = 600000

            for x in pres.slides[1:]:
                x.slide_show_transition.type = a_slides.slideshow.TransitionType.PEEL_OFF
                x.slide_show_transition.value.direction = 1
                x.slide_show_transition.advance_on_click = False
                x.slide_show_transition.speed = a_slides.slideshow.TransitionSpeed.
                x.slide_show_transition.advance_after_time = 6000


            ### Write the presentation to disk ###
            pres.save(slide_show, a_slides.export.SaveFormat.PPTX) 

    return                      

file_paths = file_path_list()
design_slide_show()

In [ ]:
%%time


def create_video(): ### creating a short video from each slideshow ###
    powerpoint = win32com.client.Dispatch("Powerpoint.Application")

    for slide_show in tqdm(file_paths):
        presentation = powerpoint.Presentations.Open(FileName=slide_show)
        presentation.CreateVideo(''.join([slide_show[:-5],'.mp4']),
                                 Quality = 85,     # resolution:The resolution of the slide. 480,720,1080...
                                 VertResolution = 1080
                                )
        time.sleep(45)
        presentation.Close()

    return   

create_video()